## MLE estimation of a simple macroeconomic model

You can observe time series data in an economy for the following variables $(c_t, k_t, w_t, r_t)$. The first line of this file contains variable labels. These data have 100 periods, which are quarterly (25 years). Suppose you think the data are generated by a pricess similar to the Brock and Mirman (1972). A simplified set of characterizing equations of the Brock and Mirman model are the following:
\begin{align}
c_t^{-1} - \beta E[r_{t+1}(c_{t+1})^{-1}] & = 0 && \text{(1)} \\
c_t - k_{t+1} - w_t - r_t k_t & = 0 && \text{(2)} \\
w_t - (1-\alpha)e^{z_t}k_t^\alpha & = 0 && \text{(3)} \\
r_t - \alpha e^{z_t} k_t^{\alpha - 1} & = 0 && \text{(4)} \\
z_t & = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t && \text{(5)} \\
\epsilon_t & \sim N(0,\sigma^2)
\end{align}
The variable $c_t$ is aggregate consumption in period $t$. $k_{t+1}$ is total household savings and investment in period $t$ for which they receive a return in the next period (this model assumes full depreciation of capital). The wage per unit of labor in period $t$ is $w_t$ adn the interest rate or rate of return on investment is $r_t$. Total factor productivity is $z_t$, which follows an AR(1) process. The rest of the symbols in the equations are parameters that must be estimated:
$$ (\alpha, \beta, \rho, \mu, \sigma) $$
The constraints on these parameters are:
$$ \alpha, \beta \in (0,1), \quad \mu,\sigma > 0, \quad \rho \in (-1,1) $$
Assume that the first observation in the data file is $t=1$. Let $k_1$ be the first observation in the data file for $k_t$. Assume that $z_0 = \mu$, so that $z_1 = \mu$. Assume that the discount factor is known to be $\beta = 0.99$.

In [78]:
# Load Modules
import numpy as np
import scipy.stats as sts
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.special as spc
import scipy.integrate as integrate
import distributions as dst

In [79]:
# Import Data
macroseries = pd.read_csv('MacroSeries.csv')

(a) Use the data $(w_t,k_t)$ and equations (3) and (5) to estimate the four parameters $(\alpha, \rho, \mu, \sigma)$ by maximum likelihood. Given a guess for the parameters $(\alpha, \rho, \mu, \sigma)$, you can use the two variables from the data $(w_t,k_t)$ and (3) to back out a series for $z_t$. You can then use equation (5) to compute the probability of each 
$$ z_t \sim N(\rho z_{t-1} + (1-\rho) \mu, \sigma^2) $$
The maximum likelihood estimate $(\hat{\alpha},\hat{\rho},\hat{\mu},\hat{\sigma})$ maximzies the likelihood function of that normal distribution of $z_t$'s. Report your estimates and the inverse hessian variance-covariance matrix of your estimates.

In [80]:
# Define z_t using (3)
def z_t(w_t,k_t,alpha):
    return np.log(w_t/((1-alpha)*k_t**alpha))

def epsilon_t(z_t, rho,mu):
    z_t_1 = z_t.shift()
    z_t_1[0] = mu
    return z_t - rho*z_t_1 - (1-rho)*mu

# Define log likelihood function
def log_lik_norm(w_t,k_t, alpha, rho, mu, sigma):
    epsilonvals = epsilon_t(z_t(w_t,k_t,alpha), rho,mu)
    ln_pdf_vals = np.log(dst.norm_pdf(epsilonvals,0,sigma))
    #ln_pdf_vals = -((epsilonvals)**2)/(2*sigma**2) - (1/2)*np.log(2*np.pi*sigma**2)
    #ln_pdf_vals = np.log(sts.norm.pdf(epsilonvals,loc=0,scale=sigma))
    return ln_pdf_vals.sum()

# Define Objective Function
def crit(params, w_t, k_t):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_norm(w_t,k_t, alpha, rho, mu, sigma)
    return -log_lik_val

### Various Tests
#epsilon_t(z_t(macroseries["w_t"],macroseries["k_t"],alpha_0), rho_0,mu_0)[1:]
#log_lik_norm(macroseries["w_t"],macroseries["k_t"], alpha_0, rho_0, mu_0, sigma_0)
#crit(np.array([alpha_0, rho_0, mu_0, sigma_0]), macroseries["w_t"],macroseries["k_t"])
#macroseries["w_t"],macroseries["k_t"]
#z_t(macroseries["w_t"],macroseries["k_t"],.5), z_t(macroseries["w_t"],macroseries["k_t"],.5).shift()
#check = z_t(macroseries["w_t"],macroseries["k_t"],.5).shift()
#check[0] = .5
#print(check)
crit(np.array([0.911802782775,  0.999999999999,  1.03857989199, 1.04649469512]), macroseries["w_t"],macroseries["k_t"])

101.3667071645704

In [81]:
# Run Maximum Likelihood with Gamma
alpha_0, rho_0, mu_0, sigma_0 = .5, 0, .5, .5
#alpha_0, rho_0, mu_0, sigma_0 = 0.46087855,  0.71646589,  9.47567957,  0.09206664
params_init = np.array([alpha_0, rho_0, mu_0, sigma_0])
mle_args = (macroseries["w_t"],macroseries["k_t"])
results1 = opt.minimize(crit, params_init, args=mle_args, method = "L-BFGS-B",
                        bounds = ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)), options ={'ftol': 1e-10})
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results1.x


In [82]:
# Print Results
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE, ' sigma_MLE=', sigma_MLE)
print(results1)

#print(z_t(macroseries["w_t"],macroseries["k_t"],alpha_MLE))

# Get Variance Covariance Matrix
vcv_mle1= results1.hess_inv.todense()
print('VCV(MLE) = ', vcv_mle1)

alpha_MLE= 0.92199196448  rho_MLE= 0.99999999 mu_MLE= 0.8384721751  sigma_MLE= 0.379922805263
      fun: 36.06331761577703
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-64.24490593,   3.13426369, -22.37210239,  47.65081698])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 105
      nit: 12
   status: 0
  success: True
        x: array([ 0.92199196,  0.99999999,  0.83847218,  0.37992281])
VCV(MLE) =  [[ 0.01692003 -0.00321064  0.00279934 -0.02074715]
 [-0.00321064  0.01679452 -0.05572713  0.02770172]
 [ 0.00279934 -0.05572713  1.0453373  -0.44003269]
 [-0.02074715  0.02770172 -0.44003269  0.21956802]]


In [83]:
minimizer_kwargs = dict(method="L-BFGS-B", 
                        bounds= ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)),
                        args = mle_args)

results1b = opt.basinhopping(crit, params_init, 
                             minimizer_kwargs = minimizer_kwargs, niter=1000)
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results1b.x
results1b

                        fun: -96.62961744707427
 lowest_optimization_result:       fun: -96.62961744707427
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00296865, -0.00043769,  0.00017053, -0.00170814])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 190
      nit: 27
   status: 0
  success: True
        x: array([ 0.46088818,  0.71644956,  9.47554935,  0.09206729])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 302
                       nfev: 82025
                        nit: 1000
                          x: array([ 0.46088818,  0.71644956,  9.47554935,  0.09206729])

(b) Use the data $(r_t,k_t)$ and equations (4) and (5) to estimate the four parameters $(\alpha, \rho, \mu, \sigma)$ by maximum likelihood. Given a guess for the parameters $(\alpha, \rho, \mu, \sigma)$, you can use the two variables from the data $(r_t,k_t)$ and (4) to back out a series for $z_t$. You can then use equation (5) to compute the probability of each 
$$ z_t \sim N(\rho z_{t-1} + (1-\rho) \mu, \sigma^2) $$
The maximum likelihood estimate $(\hat{\alpha},\hat{\rho},\hat{\mu},\hat{\sigma})$ maximzies the likelihood function of that normal distribution of $z_t$'s. Report your estimates and the inverse hessian variance-covariance matrix of your estimates.

In [84]:
# Define z_t using (4)
def z_t2(r_t,k_t,alpha):
    return np.log(r_t/(alpha*k_t**(alpha-1)))

# Same as earlier
def epsilon_t(z_t, rho,mu):
    z_t_1 = z_t.shift()
    z_t_1[0] = mu
    return z_t - rho*z_t_1 - (1-rho)*mu

# Define log likelihood function
def log_lik_norm2(r_t,k_t, alpha, rho, mu, sigma):
    epsilonvals = epsilon_t(z_t2(r_t,k_t,alpha), rho,mu)
    #ln_pdf_vals = np.log(dst.norm_pdf(epsilonvals,0,sigma))
    #ln_pdf_vals = -((epsilonvals)**2)/(2*sigma**2) - (1/2)*np.log(2*np.pi*sigma**2)
    ln_pdf_vals = np.log(sts.norm.pdf(epsilonvals,loc=0,scale=sigma))
    return ln_pdf_vals.sum()

# Define Objective Function
def crit2(params, r_t, k_t):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_norm2(r_t,k_t, alpha, rho, mu, sigma)
    return -log_lik_val

### Various Tests
#epsilon_t(z_t2(macroseries["r_t"],macroseries["k_t"],alpha_0), rho_0,mu_0)[1:]
#log_lik_norm2(macroseries["r_t"],macroseries["k_t"], alpha_0, rho_0, mu_0, sigma_0)
#crit(np.array([alpha_0, rho_0, mu_0, sigma_0]), macroseries["r_t"],macroseries["k_t"])
#macroseries["r_t"],macroseries["k_t"]
crit(np.array([0.9999999999,  0.849635638864,  1.01055911652, 0.863646623473]), macroseries["w_t"],macroseries["k_t"])

1160.9851792952347

In [85]:
# Run Maximum Likelihood with Gamma
alpha_0, rho_0, mu_0, sigma_0 = 0.5, 0.3, .5, .5
params_init2 = np.array([alpha_0, rho_0, mu_0, sigma_0])
mle_args2 = (macroseries["r_t"],macroseries["k_t"])
results2 = opt.minimize(crit2, params_init2, args=mle_args2, method = "L-BFGS-B",
                        bounds = ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)))
alpha_MLE2, rho_MLE2, mu_MLE2, sigma_MLE2 = results2.x

In [86]:
# Print Results
print('alpha_MLE2=', alpha_MLE2, ' rho_MLE2=', rho_MLE2, 'mu_MLE2=', mu_MLE2, ' sigma_MLE2=', sigma_MLE2)
print(results2)

# Get Variance Covariance Matrix
vcv_mle2 = results2.hess_inv
print('VCV(MLE) = ', vcv_mle2)

alpha_MLE2= 0.974533699532  rho_MLE2= 0.99999999 mu_MLE2= 0.510458281415  sigma_MLE2= 0.118906329237
      fun: -68.60273457267111
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 203.1278413 ,   52.29413205,    8.97118895,  -41.13039296])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 100
      nit: 12
   status: 0
  success: True
        x: array([ 0.9745337 ,  0.99999999,  0.51045828,  0.11890633])
VCV(MLE) =  <4x4 LbfgsInvHessProduct with dtype=float64>


In [87]:
minimizer_kwargs = dict(method="L-BFGS-B", 
                        bounds= ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)),
                        args = mle_args2)

results2b = opt.basinhopping(crit2, params_init2, 
                             minimizer_kwargs = minimizer_kwargs, niter=1000)

results2b

                        fun: -96.555128620383499
 lowest_optimization_result:       fun: -96.555128620383499
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.10444836,  0.0004249 , -0.0062002 , -0.03556409])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 360
      nit: 42
   status: 0
  success: True
        x: array([ 0.7198794 ,  0.45627863,  4.7677259 ,  0.09213446])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 449
                       nfev: 90935
                        nit: 1000
                          x: array([ 0.7198794 ,  0.45627863,  4.7677259 ,  0.09213446])

(c) According to your estimates from part (a), if investment/savings today is $k_t = 7,500,000$ and the productivity shock in the previous period is $z_{t-1} = 10$, what is the probability that the interest rate this period will be greater than $r_t =1$. That is, solve for $Pr(r_t > 1 | \hat{\theta},k_t, z_{t-1})$. [HINT: Use equation (4) to solve for the $z_t = z^*$ such that $r_t = 1$. Then use (5) to solve for the probability that $z_t > z^*$]

In [88]:
z_star = z_t2(1,7500000,alpha_MLE)
print('z_star = ',z_star)
z_t_minus1 = 10
Pr_r_t = 1 - integrate.quad(lambda x: 
                            dst.norm_pdf(x, rho_MLE*z_t_minus1 + (1-rho_MLE)*mu_MLE, sigma_MLE**2)
                            , 0, z_star)[0]
print('Probability that r_t will be greater than 1:', Pr_r_t)

z_star =  9.30896287249
Probability that r_t will be greater than 1: 1.0
